# **ETL 1 - General**
Se cargan las librerias y archivos necesarios

In [1]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import ast
from sklearn.preprocessing import LabelEncoder # type: ignore
import pickle

In [2]:
movies = pd.read_csv(r'C:\Users\paula\OneDrive\Curso\Henry\Proyecto Individual\Movies\movies_dataset.csv', low_memory=False)
credits = pd.read_csv(r'C:\Users\paula\OneDrive\Curso\Henry\Proyecto Individual\Movies\credits.csv', low_memory=False)

## **Procesamiento del Dataset `movies_dataset.csv`**

1. **Cálculo del Retorno de Inversión (`return`)**:

In [3]:
# Asegurar que 'revenue' y 'budget' son numéricos y reemplazar nulos por 0
movies['revenue'] = pd.to_numeric(movies['revenue'], errors='coerce').fillna(0)
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce').fillna(0)

# Calcular 'return'
movies['return'] = np.where(movies['budget'] == 0, 0, movies['revenue'] / movies['budget'])

# Reemplazar divisiones por cero o valores nulos directamente
movies['return'] = movies['return'].replace([np.inf, np.nan], 0)


2. **Procesamiento de Fechas (`release_date`)**:

In [4]:
# Convertir release_date al formato datetime y manejar errores
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

# Crear la columna release_year extrayendo el año
movies['release_year'] = movies['release_date'].dt.year

# Crear la columna release_month extrayendo el mes
movies['release_month'] = movies['release_date'].dt.month

# Crear la columna release_dow extrayendo el dia de la semana
movies['release_dow'] = movies['release_date'].dt.day_of_week

# Eliminar aquellos registros que quedaron nulos
movies = movies[movies['release_year'].notnull()]

3. **Extracción de Colecciones (`belongs_to_collection`)**:

In [5]:
# Asegurar que los valores de la columna sean tratados como diccionarios
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

# Extraer el nombre de la colección
movies['collection'] = movies['belongs_to_collection'].apply(
    lambda x: x['name'] if isinstance(x, dict) and 'name' in x else None
)

4. **Codificación de Géneros (`genres`)**:

In [6]:
# Asegurar que los valores en la columna 'genres' sean listas de diccionarios
movies['genres'] = movies['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer todos los nombres únicos de los géneros
all_genres = set()
for genre_list in movies['genres']:
    all_genres.update(genre['name'] for genre in genre_list)

# Crear el LabelEncoder y ajustarlo con los nombres únicos de los géneros
le = LabelEncoder()
le.fit(list(all_genres))

# Crear una nueva columna con los géneros codificados
movies['genres_encoded'] = movies['genres'].apply(
    lambda genre_list: le.transform([genre['name'] for genre in genre_list]).tolist()
)

5. **Eliminación de Columnas Irrelevantes para el proyecto**:

In [7]:
movies = movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'spoken_languages', 'production_countries', 'belongs_to_collection', 'production_companies', 'original_language','tagline', 'genres', 'overview', 'runtime', 'status'], errors='ignore')

6. **Normalización y Eliminación de Duplicados**:

In [8]:
movies['popularity'] = pd.to_numeric(movies['popularity'], errors='coerce')


In [9]:
# Eliminar duplicados de "title" conservando el registro con mayor popularidad
movies = movies.sort_values(by='popularity', ascending=False)  # Ordenar por popularidad
movies = movies.drop_duplicates(subset=['title'], keep='first')  # Eliminar duplicados conservando el primero (el de mayor popularidad)

7. **Guardado del Dataset Transformado**:

In [10]:
# Guardar el dataset transformado
movies.to_csv(r'../Datasets/transformed_movies.csv', index=False)

## **Procesamiento del Dataset `credits.csv`**

1. **Extracción de Directores (`crew`)**:

In [11]:
# Convertir la columna `crew` en listas de diccionarios
credits['crew'] = credits['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer el nombre de los directores
credits['director_name'] = credits['crew'].apply(
    lambda x: next((person['name'] for person in x if person.get('job') == 'Director'), None)
)

2. **Extracción de Actores (`cast`)**:

In [ ]:
# Convertir la columna `cast` en listas de diccionarios
credits['cast'] = credits['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer los nombres de todos los actores en una lista
credits['actors'] = credits['cast'].apply(
    lambda x: [actor['name'] for actor in x] if isinstance(x, list) else []
)

3. **Crear y guardar un Dataset Reducido**:

In [ ]:
# Crear un nuevo DataFrame reducido
reduced_credits = credits[['id', 'director_name', 'actors']]

# Guardar el dataset reducido
reduced_credits.to_csv(r'../Datasets/reduced_credits.csv', index=False)